<h1  align="center"> <b>   Facial Emotion Recongintion </b></h1>
<h4 align="center">By Sharoon Yaqub</h4>

In [ ]:
# Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
import tqdm
from PIL import Image
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

import os

In [ ]:
# Data Loading
PATH = "/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge"
data = pd.read_csv(os.path.join(PATH, "icml_face_data.csv"))
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
print(data)

In [ ]:
# Function to parse data into right format
# Output: Image in right shaped and normalized + labels
def parse_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48, 1))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48, 1))
        image_array[i] = image
        
    return image_array, image_label

# Splitting the data into train, validation and testing set thanks to Usage column
train_imgs, train_lbls = parse_data(data[data[" Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data[" Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data[" Usage"] == "PublicTest"])

In [ ]:
print("train shape", np.shape(train_imgs))
print("validation shape", np.shape(val_imgs))
print("validatio shape", np.shape(val_imgs))

In [ ]:
import os, shutil 
os.mkdir("/kaggle/working/imgs")
data = np.array(train_imgs[:5])
i = 0
for px_map in data:
    i = i + 1
    px_map = np.reshape(px_map, (48, 48))
    image = Image.fromarray(px_map)
    image = image.convert('RGB')
    image.save('/kaggle/working/imgs/'+str(i)+'.bmp')

In [ ]:
# Define the number of splits for k-fold cross-validation
n_splits = 5
# Define lists to store the loss and accuracy for each fold
loss_per_fold = []
acc_per_fold = []
loss_test_fold = []
accuracy_test_fold = []
# Define the k-fold cross-validator
kfold = KFold(n_splits=n_splits, shuffle=True)

In [ ]:
history_list = []
# Loop over the k folds
for fold, (train_indices, val_indices) in enumerate(kfold.split(train_imgs, train_lbls)):

    # Print the fold number
    print(f'Fold {fold+1}/{n_splits}:')

    # Split the data into training and validation sets
    x_train, y_train = train_imgs[train_indices], train_lbls[train_indices]
    x_val, y_val = train_imgs[val_indices], train_lbls[val_indices]

    # Define the CNN model
    model_cnn = models.Sequential()
    model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model_cnn.add(layers.MaxPooling2D((2, 2)))
    model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model_cnn.add(layers.MaxPooling2D((2, 2)))
    model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model_cnn.add(layers.MaxPooling2D((2, 2)))
    model_cnn.add(layers.Flatten())
    model_cnn.add(layers.Dense(128, activation='relu'))
    model_cnn.add(layers.Dropout(0.5))
    model_cnn.add(layers.Dense(7, activation='softmax'))
    
    # Compile the model
    model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history_cnn = model_cnn.fit(x_train, y_train, epochs=20, batch_size=32,
                            validation_data=(x_val, y_val), verbose=1)
    
    history_list.append(history_cnn)

    # Evaluate the model on the validation set
    loss, accuracy = model_cnn.evaluate(x_val, y_val, verbose=0)
    print(f'Validation accuracy: {accuracy*100:.2f}%')
    loss_per_fold.append(loss)
    acc_per_fold.append(accuracy*100)

    # Evaluate the model on the test set
    loss_test, accuracy_test = model_cnn.evaluate(test_imgs, test_lbls, verbose=0)
    print(f'Test accuracy: {accuracy_test*100:.2f}%')
    loss_test_fold.append(loss_test)
    accuracy_test_fold.append(accuracy_test*100)

In [ ]:
# Extract the metric values from all histories
metric_values = []
for history in history_list:
    metric_values.append(history.history['val_accuracy'])

# Calculate mean and standard deviation of the metric
mean_metric = np.mean(metric_values)
std_metric = np.std(metric_values)

print("Mean metric value:", mean_metric)
print("Standard deviation of metric:", std_metric)

In [ ]:
# Train and validation accuracy vs. epoch
train_acc = history_cnn.history['accuracy']
val_acc = history_cnn.history['val_accuracy']
epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Train and validation loss vs. epoch
train_loss = history_cnn.history['loss']
val_loss = history_cnn.history['val_loss']

plt.plot(epochs, train_loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Predict the labels of the test set
y_pred = model_cnn.predict_classes(test_imgs)

# Generate the confusion matrix
cm = confusion_matrix(test_lbls, y_pred)
print(cm)

# Plot the confusion matrix
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(emotions))
plt.xticks(tick_marks, emotions.values(), rotation=45)
plt.yticks(tick_marks, emotions.values())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
model_cnn.summary()

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
matr, ax = plot_confusion_matrix(conf_mat=cm,
                                show_normed=True,
                                show_absolute=False,
                                class_names=emotions.values(),
                                figsize=(8, 8))
matr.show()